In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from  pyspark.sql import SQLContext
from pyspark.sql.types import *


import shutil
import os

import file_creator_streamer



In [9]:
#Global
INPUT1 = "input_stream_csv1"
INPUT2 = "input_stream_csv2"
OUTPUT = "stream_dir"
CHECKPOINT_DIR="checkpoint_dir"
#2019-01-23 00:35:00,key1,1


In [10]:
def create_data():
    pass

In [11]:
def clean_dirs():
    shutil.rmtree(OUTPUT, ignore_errors=True)
    shutil.rmtree(CHECKPOINT_DIR,ignore_errors=True)
    shutil.rmtree(INPUT1)
    shutil.rmtree(INPUT2)
    os.mkdir(OUTPUT)
    os.mkdir(CHECKPOINT_DIR)
    os.mkdir(INPUT1)
    os.mkdir(INPUT2)
clean_dirs()

In [12]:
spark = SparkSession.builder \
             .master("local") \
             .appName("stream to csv")\
             .config("spark.some.config.option", "some-value") \
            .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [13]:
dataSchema = StructType([StructField('creation_time',TimestampType(),True),
        StructField('key',StringType(),True),
        StructField('x',LongType(),True),
                        ])

In [14]:
streaming1 = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.csv(INPUT1)
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [15]:
streaming2 = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.csv(INPUT2)
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [16]:
streaming1.registerTempTable("my_table1")
streaming2.registerTempTable("my_table2")


/home/henry/Envs3/pyspark_3/lib/python3.8/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [17]:
spark.sql("select t1.creation_time, t1.x as x1, t2.x as x2 from my_table1 t1 inner join my_table2 t2 on t1.key = t2.key")\
.writeStream.trigger(processingTime='10 seconds')\
.format("parquet")\
.option("checkpointLocation", CHECKPOINT_DIR).outputMode("append")\
.option("path", OUTPUT).start()

In [ ]:
#df = spark.read.parquet("stream_dir5/")
#df.registerTempTable("result")
#spark.sql("""select creation_time, current_timestamp() -
#INTERVAL 14 minutes as t from result where creation_time = '2019-01-22 00:03:49'
#and creation_time < current_timestamp() - interval 60 minutes
#""").show()


In [23]:
df = spark.read.parquet(f"{OUTPUT}/")
df.registerTempTable("result")
spark.sql("""select * from result 
""").show()

+-------------------+---+---+
|      creation_time| x1| x2|
+-------------------+---+---+
|2024-06-06 17:26:15| 35| 97|
|2024-06-06 17:27:15| 19| 97|
|2024-06-06 17:24:14|  8|  7|
|2024-06-06 17:25:14| 46|  7|
+-------------------+---+---+

